# ✈️ Airline Assistant – GenAI Capstone (Kaggle x Google)

This notebook is a submission for the **Google x Kaggle 5-Day Generative AI Event**.

built a conversational airline assistant that:
- Answers travel-related questions using embeddings and similarity search
- Classifies user input to distinguish between questions and personal details
- Collects user information across turns (name, flight number, issue, email)
- Runs through an interactive Gradio chatbot interface

You can test the assistant at the end of this notebook.

Thanks to Kaggle & Google for the challenge! 🛫


# Imports & Setup

In [1]:
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai
import numpy as np
from numpy.linalg import norm

# Load Gemini API key
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)


In [2]:
pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


#  FAQ List

In [3]:
faq_list = [
    {
        "question": "Can I bring a dog or any pet with me on the plane?",
        "answer": "Yes, pets under 8kg can be brought in the cabin if they're in a secure carrier. Larger animals must travel in the cargo hold. Advance notice is required."
    },
    {
        "question": "How much is the fee for extra or overweight baggage?",
        "answer": "Extra bags up to 23kg cost $30 each. For baggage over 23kg, overweight fees apply and vary based on your flight route."
    },
    {
        "question": "What identification do I need for airport check-in?",
        "answer": "A valid government-issued photo ID is required for domestic flights. For international travel, a passport and necessary visas are mandatory."
    },
    {
        "question": "When should I arrive at the airport before departure?",
        "answer": "You should arrive at least 2 hours before domestic flights and 3 hours before international flights to allow time for check-in and security."
    },
    {
        "question": "Can I change or modify my flight after booking?",
        "answer": "Yes, you can change your flight up to 2 hours before departure. Change fees may apply depending on your fare class."
    },
    {
        "question": "Do I need a visa to travel internationally?",
        "answer": "Visa requirements depend on your nationality and destination. Please check the destination country’s official government website for updated rules."
    },
    {
        "question": "Can I cancel my flight and get a refund?",
        "answer": "You can cancel your booking up to 24 hours before departure. Refunds depend on your ticket type and cancellation policy."
    },
    {
        "question": "Is food provided on the flight?",
        "answer": "We offer complimentary meals and snacks on international flights. For domestic routes, snacks and drinks may be available for purchase."
    },
    {
        "question": "Is Wi-Fi available during the flight?",
        "answer": "Yes, Wi-Fi is available on most international flights and select domestic routes. Charges vary depending on your plan."
    },
    {
        "question": "Can I bring snacks or food through airport security?",
        "answer": "Yes, solid foods like chips or sandwiches are allowed through security. Liquids over 100ml are not permitted."
    }
]


# Embedding Setup

In [4]:
faq_questions = [item["question"] for item in faq_list]

faq_embeddings = [
    genai.embed_content(
        model="models/text-embedding-004",
        content=q,
        task_type="retrieval_document"
    )["embedding"]
    for q in faq_questions
]


# FAQ Search Logic

In [5]:
def embed_query(text):
    result = genai.embed_content(
        model="models/text-embedding-004",
        content=text,
        task_type="retrieval_query"
    )
    return result["embedding"]

def find_best_faq_answer(user_question, threshold=0.6):
    user_vector = embed_query(user_question)
    similarities = [
        np.dot(user_vector, faq_vec) / (norm(user_vector) * norm(faq_vec))
        for faq_vec in faq_embeddings
    ]

    best_index = int(np.argmax(similarities))
    best_score = similarities[best_index]

    if best_score < threshold:
        return {
            "match_question": None,
            "answer": None,
            "similarity": round(best_score, 4)
        }

    return {
        "match_question": faq_list[best_index]["question"],
        "answer": faq_list[best_index]["answer"],
        "similarity": round(best_score, 4)
    }


# Gemini Classifier + Extractor

In [6]:
def classify_user_input_with_flash(user_input):
    prompt = f"""
You're a smart input classifier for an airline assistant.

Classify this input as:
- "faq" → general travel question
- "info" → personal detail (name, flight, or issue)

Input: "{user_input}"
Classification:"""
    model = genai.GenerativeModel("models/gemini-2.0-flash")
    response = model.generate_content(prompt)
    return response.text.strip().lower()

In [7]:
def extract_and_validate_field(field, raw_input):
    prompt = f"""
Extract the relevant value for the field: "{field}".  
If no valid value is present, return "none".

Examples:
- Field: name | Input: "My name is Firas" → Firas
- Field: confirmation_number | Input: "AF101" → AF101
- Field: issue | Input: "I lost my bag" → lost my bag
- Field: email | Input: "You can send it to firas@mail.com" → firas@mail.com
- Field: confirmation_number | Input: "sure!" → none

Input: "{raw_input}"  
Output:
"""
    model = genai.GenerativeModel("models/gemini-2.0-flash")
    response = model.generate_content(prompt)
    answer = response.text.strip()
    return None if answer.lower() in ["none", "not sure", ""] else answer


# Assistant Logic

In [8]:
def next_question(state):
    for key in ["name", "confirmation_number", "issue", "email"]:
        if state.get(key) is None:
            question = {
                "name": "👋 Welcome! May I have your name?",
                "confirmation_number": "Can you provide your flight confirmation number?",
                "issue": "What issue are you facing? (e.g. lost baggage, delay, reschedule)",
                "email": "Lastly, please enter your email to receive a summary of this conversation."
            }
            return key, question[key]
    return None, None

In [9]:
def handle_user_input(user_input, state):
    key, _ = next_question(state)
    if key is None:
        return " All information has been collected.", True

    extracted = extract_and_validate_field(key, user_input)
    if extracted is None:
        return f" I didn't catch that. Could you clarify your {key}?", False

    state[key] = extracted
    _, next_q = next_question(state)

    if next_q:
        return next_q, True
    else:
        return (
            f"Thank you! Here's a summary of your information:\n"
            f"- Name: {state['name']}\n"
            f"- Flight: {state['confirmation_number']}\n"
            f"- Issue: {state['issue']}\n"
            f"- Email: {state['email']}\n\n"
            "📧 A confirmation message will be sent to your email.", True
        )


# Hybrid Assistant (FAQ + Form Filling)

In [10]:
def hybrid_assistant(user_input, state, threshold=0.6):
    input_type = classify_user_input_with_flash(user_input)

    if input_type == "faq":
        result = find_best_faq_answer(user_input, threshold=threshold)
        if result["match_question"]:
            return f" FAQ Match:\nQ: {result['match_question']}\nA: {result['answer']}"
        else:
            return " I couldn't find a matching answer. Can I help you with your name, flight, or issue?"

    elif input_type == "info":
        response, _ = handle_user_input(user_input, state)
        return response

    else:
        return " Sorry, I couldn't understand your input type."


In [11]:
user_state = {"name": None, "confirmation_number": None, "issue": None, "email": None}
print(" Assistant reset. Current state:", user_state)
print(hybrid_assistant("Can i bring my dog?", user_state))

print(hybrid_assistant("My name is Firas Ridene", user_state))
print(hybrid_assistant("AF101", user_state))
print(hybrid_assistant("I lost my bag in Rome", user_state))
print(hybrid_assistant("Can i bring my cat?", user_state))

print(hybrid_assistant("firas.ridene@esprit.tn", user_state))
print(hybrid_assistant("Do I need a visa to fly to Italy?", user_state))

print("Final state:", user_state)


 Assistant reset. Current state: {'name': None, 'confirmation_number': None, 'issue': None, 'email': None}
 FAQ Match:
Q: Can I bring a dog or any pet with me on the plane?
A: Yes, pets under 8kg can be brought in the cabin if they're in a secure carrier. Larger animals must travel in the cargo hold. Advance notice is required.
Can you provide your flight confirmation number?
What issue are you facing? (e.g. lost baggage, delay, reschedule)
Lastly, please enter your email to receive a summary of this conversation.
 FAQ Match:
Q: Can I bring a dog or any pet with me on the plane?
A: Yes, pets under 8kg can be brought in the cabin if they're in a secure carrier. Larger animals must travel in the cargo hold. Advance notice is required.
Thank you! Here's a summary of your information:
- Name: Firas Ridene
- Flight: AF101
- Issue: lost my bag in Rome
- Email: firas.ridene@esprit.tn

📧 A confirmation message will be sent to your email.
 FAQ Match:
Q: Do I need a visa to travel internationall

In [12]:
import gradio as gr

# Reset chat state
chat_state = {
    "name": None,
    "confirmation_number": None,
    "issue": None,
    "email": None  # still collected, but no sending
}
first_turn = True  # For the welcome message

def chat_bot(message, history):
    global first_turn

    # 👋 Show welcome message at the start
    if first_turn:
        first_turn = False
        return {"role": "assistant", "content": "👋 Welcome! May I have your name?"}

    # 🧠 Route input to hybrid assistant
    response = hybrid_assistant(message, chat_state)

    # ✅ Once all info is gathered, show summary only (no email sending)
    if all(chat_state.values()):
        response += (
            f"\n\n✅ Summary of your details:\n"
            f"- Name: {chat_state['name']}\n"
            f"- Flight: {chat_state['confirmation_number']}\n"
            f"- Issue: {chat_state['issue']}\n"
            f"- Email: {chat_state['email']}"
        )

    return {"role": "assistant", "content": response}

# 🚀 Launch Gradio chat interface
gr.ChatInterface(
    chat_bot,
    title="✈️ Airline Assistant",
    type="messages"
).launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://99053acc3851e4d55d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
